# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Porciuncula Yamada

Nome: Pedro Henrique de Sousa da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

* Tweets relevantes:
    - Críticas à marca ou produtos
    - Anúncios de eventos
    - Venda de produtos


___

## Escolha do produto

* Produto escolhido: **`Apple`**

  - Apple é uma empresa norte-americana que recentemente anunciou o lançamento de diversos produtos como o iPhone 13 e o Apple Watch 7.
    A classificação foi feita considerando se o tweet tem relação com a marca ou se faz parte de uma discussão relacionada à empresa: opinião, crítica ou afim.

___

Carregando algumas bibliotecas:

Na falta de alguma das bibliotecas abaixo, descomentar a linha da biblioteca e executar a célula:

In [1]:
# !pip install pandas
# !pip install matplotlib
# !pip install numpy
# !pip install functools
# !pip install operator
# !pip install re
# !pip install emoji
# !pip install nltk
# nltk.download('stopwords')

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
import functools
import operator
import re 
import nltk

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\sjcpe\OneDrive\Área de Trabalho\C.Dados\Projeto 1\cdados-projeto-1-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'Apple.xlsx'

In [5]:
train = dft = pd.read_excel('Apple.xlsx', sheet_name='Treinamento')
test = dft = pd.read_excel('Apple.xlsx', sheet_name='Teste')

___
## Montando um Classificador Naive-Bayes

Aqui criamos funções com os seguintes intuitos:
- Remoção de pontuação (`função I: cleanup()`);
- Correção de espaços entre emojis (`função II: separa_emoji()`);
- Remoção de palavras de parada (ou *stopwords*) (`função III: remove_stopwords()`);

Ao final da montagem, também aplicamos a Suavização de Laplace.

In [6]:
# FUNÇÃO I
def cleanup(text):
    """
        Função de limpeza muito simples que troca
        alguns sinais básicos por espaços
    """
    pont = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(pont)

    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '', text_subbed)

    return text_subbed

# FUNÇÃO II
def separa_emoji(text):
    """
        Função que separa emojis em frases
    """
    em_split_emoji = emoji.get_emoji_regexp().split(text)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    return em_split

# FUNÇÃO III
def remove_stopwords(lista):
    """
        Esta função remove palavras de parada,
        tais como 'as', 'os', entre outras
    """
    stopwords = nltk.corpus.stopwords.words('portuguese')
    l = list()
    for p in lista:
        if not p in stopwords:
            l.append(p)
    return l

___
## Obtenção de valores a partir dos DataFrames

Criando uma série para contabilizar as frequências relativas de **todos os tweets** da planilha '`Treinamento`' da base de dados:

In [7]:
lista = list()
for i in range(train.shape[0]):
    lista += remove_stopwords(separa_emoji(cleanup(train.Treinamento[i])))
serie_twt = pd.Series(lista)
twt_relat = serie_twt.value_counts(True)

Elaborando tabelas separadas para os tweets classificados como '`relevantes`' ou '`irrelevantes`':

In [8]:
bool_ = train.isin([1])

filtro_r = bool_['Classificação'] == True
filtro_i = bool_['Classificação'] == False

relevantes = train.loc[filtro_r, :]
irrelevantes = train.loc[filtro_i, :]

Criando uma série para contabilizar as frequências relativas de **tweets relevantes** e de **tweets irrelevantes** da planilha '`Treinamento`':

In [9]:
li_rel = list()
li_irr = list()

for i in range(relevantes.shape[0]):
    li_rel += remove_stopwords(separa_emoji(cleanup(relevantes.Treinamento[relevantes.index[i]])))

for i in range(irrelevantes.shape[0]):
    li_irr += remove_stopwords(separa_emoji(cleanup(irrelevantes.Treinamento[irrelevantes.index[i]])))

serie_rel = pd.Series(li_rel)
rel_relat = serie_rel.value_counts(True)

serie_irr = pd.Series(li_irr)
irr_relat = serie_irr.value_counts(True)

Criando uma nova coluna na planilha '`Teste`' da base de dados, onde guardaremos os valores `1` ou `0` a partir da análise feita pelo classificador:

In [10]:
test['SemLaplace'] = ''

## Classificador - Teorema de Bayes

O parâmetro usado pelo classificador será a comparação da probabilidade de o tweet ser ou não relevante. Isto é, sendo $P(R)$ e $P(I)$ a probabilidade de o tweet ser relevante ou irrelevante, respectivamente, temos:

- $P(R|tweet) > P(I|tweet)$, o tweet é relevante;
- $P(I|tweet) > P(R|tweet)$, o tweet é irrelevante;

Sabendo que $P(R|tweet) = \frac{P(tweet|R)P(R)}{P(tweet)}$ e que $P(I|tweet) = \frac{P(tweet|I)P(I)}{P(tweet)}$, a partir do `Teorema de Naive-Bayes`, podemos calcular $P(tweet|R)$ e $P(tweet|I)$ por:

- $P(tweet|R) = P(palavra_1|R).P(palavra_2|R).P(palavra_3|R)...P(palavra_n|R)$
- $P(tweet|I) = P(palavra_1|I).P(palavra_2|I).P(palavra_3|I)...P(palavra_n|I)$

Considerando essas afirmações e o fato de que não precisamos de $P(tweet)$ para realizarmos a comparação (denominadores iguais), podemos escrever o seguinte código:

In [11]:
probR = len(li_rel)/len(lista)
probI = len(li_irr)/len(lista)

for n in range(test.shape[0]):
    probTdadoI = 1
    probTdadoR = 1

    tweet = remove_stopwords(separa_emoji(cleanup(test.Teste[n])))

    for p in tweet:
        if p in rel_relat:
            probTdadoR *= rel_relat[p]
    
    for p in tweet:
        if p in irr_relat:
            probTdadoI *= irr_relat[p]
    
    probRdadoT = probTdadoR * probR
    probIdadoT = probTdadoI * probI

    if probRdadoT > probIdadoT:
        test.SemLaplace[n] = 1
    else:
        test.SemLaplace[n] = 0

<ipython-input-11-903f1c67b194>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.SemLaplace[n] = 1
<ipython-input-11-903f1c67b194>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.SemLaplace[n] = 0


## Suavização de Laplace

Vamos agora implementar a suavização de Laplace, que consiste em incluir os casos que ocorrem quando alguma palavra não está na nossa base de dados, o que anteriormente não eram consideradas no calculo. 

Mas como ela funciona? Bem, através de um equacionamento em que é somado uma unidade ao numero de frequência absoluta de vezes em que a palavras apareceu no denominador, retirando a possibilidade de resultar em 0. Como na fórmula descrita abaixo:

$$P(palavra1|I) = \frac{F_{AI}+1}{P_{I}+P_p}$$

Onde:

$ F_{AR}$: Frequência absoluta da palavra na categoria relevante

$ F_{AI}$: Frequência absoluta da palavra na categoria irrelevante

$P_{R}$: Todas as palavras pertencentes aos tweets rotulados como relevantes

$P_{I}$: Todas as palavras pertencentes aos tweets rotulados como irrelevantes

$P_p$: Todas as palavras possíveis na base de dados de treinamento





Agora vamos aplicar esse algoritmo no nosso projeto!

In [12]:
# Guarda as frequências absolutas nas variáveis
palavras_possiveis = serie_twt.value_counts().shape[0]
qtd_rel = serie_rel.value_counts().shape[0]
qtd_irr = serie_irr.value_counts().shape[0]

In [13]:
rel_abs = serie_rel.value_counts()
irr_abs = serie_irr.value_counts()

In [14]:
# Cria mais uma coluna para salvar a probabilidade utilizando a suavização de laplace
test['Laplace'] = ''

In [15]:
# Realiza o equacionamento para calcular a probabilidade 
for n in range(test.Teste.shape[0]):
    produtoR = 1
    tweet = remove_stopwords(separa_emoji(cleanup(test.Teste[n])))
    for p in tweet:
        if not p in rel_abs:
            rel_abs[p] = 0
        produtoR *= (rel_abs[p] + 1)/(qtd_rel + palavras_possiveis)

    produtoI = 1

    for p in tweet:
        if not p in irr_abs:
            irr_abs[p] = 0
        produtoI *= (irr_abs[p] + 1)/(qtd_irr + palavras_possiveis)

    probRdadoT = produtoR * probR
    probIdadoT = produtoI * probI

    # Guarda o resultado na coluna 

    if probRdadoT > probIdadoT:
        test.Laplace[n] = 1
    else:
        test.Laplace[n] = 0

<ipython-input-15-8220195304f6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Laplace[n] = 1
<ipython-input-15-8220195304f6>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Laplace[n] = 0


___

## Verificação de performance

 Agora, testamos a qualidade do nosso quassificador a partir de alguns percentuais, nas duas situações a seguir:

* **Sem a Suavização de Laplace**

In [16]:
b = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.SemLaplace[i]:
        b += 1

print(f'- Exatidão: {b/test.shape[0]*100}%')
print(f'- Porcentagem de erro: {(1 - b/test.shape[0])*100:.1f}%')

# PARA VERDADEIROS POSITIVOS
vpC = test.Classificação == 1
vpN = test.SemLaplace == 1

# PARA FALSOS POSITIVOS
fpC = test.Classificação == 0
fpN = test.SemLaplace == 1

# PARA VERDADEIROS NEGATIVOS
vnC = test.Classificação == 0
vnN = test.SemLaplace == 0

# PARA FALSOS NEGATIVOS
fnC = test.Classificação == 1
fnN = test.SemLaplace == 0

vp = (test.Teste.loc[vpC & vpN].count()/vpC.sum())*100
fp = (test.Teste.loc[fpC & fpN].count()/fpC.sum())*100
vn = (test.Teste.loc[vnC & vnN].count()/vnC.sum())*100
fn = (test.Teste.loc[fnC & fnN].count()/fnC.sum())*100

print(f'- Verdadeiros positivos (tweets relevantes classificados como relevantes): {vp:.1f}%\n'
      f'- Falsos positivos (tweets irrelevantes classificados como relevantes): {fp:.1f}%\n'
      f'- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): {vn:.1f}%\n'
      f'- Falsos negativos (tweets relevantes classificados como irrelevantes): {fn:.1f}%\n')



- Exatidão: 46.5%
- Porcentagem de erro: 53.5%
- Verdadeiros positivos (tweets relevantes classificados como relevantes): 41.0%
- Falsos positivos (tweets irrelevantes classificados como relevantes): 48.0%
- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): 52.0%
- Falsos negativos (tweets relevantes classificados como irrelevantes): 59.0%



In [17]:
# EM RELAÇÃO AO TOTAL
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.SemLaplace[i]:
        a += 1

print(f'- Exatidão: {a/test.shape[0]*100}%')
print(f'- Porcentagem de erro: {(1 - a/test.shape[0])*100:.1f}%')

# VERDADEIROS POSITIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 1 and test.SemLaplace[i] == 1:
        a += 1
print(f'- Verdadeiros positivos (relevantes): {a/test.shape[0]*100}%')

# FALSOS POSITIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 0 and test.SemLaplace[i] == 1:
        a += 1
print(f'- Falsos positivos (relevantes): {a/test.shape[0]*100:.1f}%')

# VERDADEIROS NEGATIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 0 and test.SemLaplace[i] == 0:
        a += 1
print(f'- Verdadeiros negativos (irrelevantes): {a/test.shape[0]*100}%')

# FALSOS NEGATIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 1 and test.SemLaplace[i] == 0:
        a += 1
print(f'- Falsos negativos (irrelevantes): {a/test.shape[0]*100}%')

- Exatidão: 46.5%
- Porcentagem de erro: 53.5%
- Verdadeiros positivos (relevantes): 20.5%
- Falsos positivos (relevantes): 24.0%
- Verdadeiros negativos (irrelevantes): 26.0%
- Falsos negativos (irrelevantes): 29.5%


* **Com a Suavização de Laplace**

In [18]:
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.Laplace[i]:
        a += 1

print(f'- Exatidão: {a/test.shape[0]*100}%')
print(f'- Porcentagem de erro: {(1 - a/test.shape[0])*100:.1f}%')

# PARA VERDADEIROS POSITIVOS
vpC = test.Classificação == 1
vpL = test.Laplace == 1

# PARA FALSOS POSITIVOS
fpC = test.Classificação == 0
fpL = test.Laplace == 1

# PARA VERDADEIROS NEGATIVOS
vnC = test.Classificação == 0
vnL = test.Laplace == 0

# PARA FALSOS NEGATIVOS
fnC = test.Classificação == 1
fnL = test.Laplace == 0

vp = (test.Teste.loc[vpC & vpL].count()/vpC.sum())*100
fp = (test.Teste.loc[fpC & fpL].count()/fpC.sum())*100
vn = (test.Teste.loc[vnC & vnL].count()/vnC.sum())*100
fn = (test.Teste.loc[fnC & fnL].count()/fnC.sum())*100

print(f'- Verdadeiros positivos (tweets relevantes classificados como relevantes): {vp:.1f}%\n'
      f'- Falsos positivos (tweets irrelevantes classificados como relevantes): {fp:.1f}%\n'
      f'- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): {vn:.1f}%\n'
      f'- Falsos negativos (tweets relevantes classificados como irrelevantes): {fn:.1f}%\n')


- Exatidão: 69.5%
- Porcentagem de erro: 30.5%
- Verdadeiros positivos (tweets relevantes classificados como relevantes): 96.0%
- Falsos positivos (tweets irrelevantes classificados como relevantes): 57.0%
- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): 43.0%
- Falsos negativos (tweets relevantes classificados como irrelevantes): 4.0%



In [19]:
# EM RELAÇÃO AO TOTAL
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.Laplace[i]:
        a += 1

print(f'- Exatidão: {a/test.shape[0]*100}%')
print(f'- Porcentagem de erro: {(1 - a/test.shape[0])*100:.1f}%')

# VERDADEIROS POSITIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 1 and test.Laplace[i] == 1:
        a += 1
print(f'- Verdadeiros positivos (relevantes): {a/test.shape[0]*100}%')

# FALSOS POSITIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 0 and test.Laplace[i] == 1:
        a += 1
print(f'- Falsos positivos (relevantes): {a/test.shape[0]*100:.1f}%')

# VERDADEIROS NEGATIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 0 and test.Laplace[i] == 0:
        a += 1
print(f'- Verdadeiros negativos (irrelevantes): {a/test.shape[0]*100}%')

# FALSOS NEGATIVOS
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == 1 and test.Laplace[i] == 0:
        a += 1
print(f'- Falsos negativos (irrelevantes): {a/test.shape[0]*100}%')

- Exatidão: 69.5%
- Porcentagem de erro: 30.5%
- Verdadeiros positivos (relevantes): 48.0%
- Falsos positivos (relevantes): 28.5%
- Verdadeiros negativos (irrelevantes): 21.5%
- Falsos negativos (irrelevantes): 2.0%


___
### Concluindo

### Comparando os percentuais 

Analisando os dados obtidos a partir dos cálculos acima, temos que:

A exatidão/porcentagem de erro são dois parâmetros complementares que nos ajudam a perceber a qualidade de nossa classificação de modo geral. 

* Sem a suavização de Laplace percebemos que a porcentagem de erro, que foi calculada a partir da exatidão do classificador ($erro = 1 - exatidão$), foi razoavelmente alta, indicando que nessa primeira situação o nosso classificador ainda poderia ser aperfeiçoado;

* Com a suavização de Laplace percebemos que a porcentagem de erro diminiu, o que nos levou a concluir que a qualidade do classificador está mais adequada.

As porcentagem de verdadeiros e falsos positivos/negativos são outros indicadores que explicitam a qualidade do nosso classificador. Assim temos:

* Em relação ao total de classificações (relevantes + irrelevantes), ao total de tweets classificados como relevantes e ao total de tweets classificados como irrelevantes:

    - Sem a suavização de Laplace, podemos observar que a porcentagem de classificações feitas de forma incorreta é relativamente alta quando comparada a outras porcentagens (verdadeiros posivos/negativos);

    - Com a suavização de Laplace, obtivemos uma porcentagem de verdadeiros positivos/negativos mais adequada para mostrar que, nessa situação, o nosso classificador está mais sofisticado.


### Mensagens com dupla negação e sarcasmo 

O nosso classificador não compreende tweets que contanham sarcasmo ou negação, pois a probabilidade é calculado com base nas palavras que compõe o tweet, ou seja, o tweet será classificado como relevante ou irrelevante independentemente da presença de dupla negação ou sarcasmo.

### Por que não usar o próprio classificador para gerar mais amostras de treinamento

O classificador foi construído a partir de uma base de dados contendo os tweets para treinamento. Na possibilidade de uma realimentação da base de dados utilizando a mesma base, gerará um viés que fará com que os novos dados sejam classificados de forma inadequada. 

### Novos cenários para o classificador

*  Recomendação de produtos com base no histórico de pesquisa do usúario;
*  Para a separação de email tomando como base o seu conteúdo, por exemplo, classificar o email como spam;
*  O corretor automático de digitação, o qual utiliza uma base de dados das palavras que mais digita.

### Possíveis melhorias

Poderiam ser usados outros métodos disponíveis na biblioteca `NLTK`, como a lemmatização e a deriavação de palavras (lemmatization e stemming, respectivamente), além de aplicar outras técnicas de limpeza nos tweets. Também podemos aumentar nossa base de dados da planilha treinamento, para aumentar a quantidade de informações que o classificador possui para classificar o tweet como relevante ou irrelevante.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

___

## Referências usadas no Projeto

- [Biblioteca NLTK](https://www.nltk.org)

- [Biblioteca Emoji](https://pypi.org/project/emoji/)